In [1]:
import dask.bag as db
import dask.array as da
import dask.dataframe as df
import math
import pandas as pd
import numpy as np
import requests

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


Client Scheduler: tcp://127.0.0.1:33759 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 33.68 GB


Helper function that displays all the functions that can be invoked on an object.

In [3]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [4]:
def get_split_words(word_bag):    
    split_words = word_bag.str.split()
    stripped_words = split_words.flatten().map(lambda x: x.strip())
    word_array = stripped_words.filter(lambda x: x!= "")
    return word_array

In [5]:
def get_filtered_words(words):
    stopwords = open("/home/dsp_team_gregory/gregory-p1/data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [6]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    split_words = get_split_words(word_bag)
    long_words = split_words.filter(lambda x: len(x) < 3)
#     words_without_punctuations = long_words.map(puntuation_stripper)
    return get_filtered_words(long_words)    
#     return long_words

In [7]:
def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    large_bag = db.concat(unique_words)
    frequencies = large_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], round(math.log((len(words_by_document) + 1)/x[1]), 10)))
    return idf

In [8]:
def get_TF_IDF(tf, idf):
    tf = np.asarray(tf.frequencies().compute())
    tf_idf = {}
    for idf_item in idf: 
        col_name = str(int(idf_item[0], 16))
        tf_idf[col_name] = idf_item[1]
        for i in range(len(tf)):
            tf_item = tf[i]
            if idf_item[0] == tf_item[0]:
                tf_idf[col_name] = float(idf_item[1]) * float(tf_item[1])
                np.delete(tf, i)
                break
    return tf_idf

In [9]:
set_name = "X_small_test.txt"
# set_name = "X_train.txt"
# url = "/home/dsp_team_gregory/gregory-p1/data/train/" + set_name
url = 'https://storage.googleapis.com/uga-dsp/project1/files/' + set_name
file_names = pd.read_csv(url, header=None)[0].to_numpy()

In [10]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

04:33:31


In [11]:
all_file_words = []
for file_name in file_names:
#     file_name = "/home/dsp_team_gregory/gregory-p1/data/train/" + file_name + ".bytes"
    file_name = "https://storage.googleapis.com/uga-dsp/project1/data/bytes/" + file_name + ".bytes"
    all_file_words.append(get_words_for_document(file_name))
idf = get_IDF(all_file_words)

In [12]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

04:33:34


In [13]:
idf = np.asarray(idf.topk(256, key=0).compute())

In [14]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

04:39:30


In [15]:
cols = np.arange(255, -1, -1).astype(str)
pd_df = pd.DataFrame(columns=cols)
i = 0
for file_words in all_file_words:
    print("working on file number %d" % i)
    i += 1
    tf_idf_bag_array = get_TF_IDF(file_words, idf)
    pd_df = pd_df.append(tf_idf_bag_array, ignore_index=True)

working on file number 0
working on file number 1
working on file number 2
working on file number 3
working on file number 4
working on file number 5
working on file number 6
working on file number 7
working on file number 8
working on file number 9
working on file number 10
working on file number 11
working on file number 12
working on file number 13
working on file number 14
working on file number 15
working on file number 16
working on file number 17
working on file number 18
working on file number 19
working on file number 20
working on file number 21
working on file number 22
working on file number 23
working on file number 24
working on file number 25
working on file number 26
working on file number 27
working on file number 28
working on file number 29
working on file number 30
working on file number 31
working on file number 32
working on file number 33
working on file number 34
working on file number 35
working on file number 36
working on file number 37
working on file number

In [16]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

04:42:21


In [17]:
csv_name = "/home/dsp_team_gregory/gregory-p1/data/" + set_name + ".csv"
pd_df.to_csv(csv_name)
csv_name

'/home/dsp_team_gregory/gregory-p1/data/X_small_test.txt.csv'

In [ ]:
tf_idf_df = df.from_pandas(pd_df, npartitions=1)
final_dataframe = tf_idf_df.compute()

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

22:38:57


In [ ]:
final_dataframe

,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,1.49871,1.94561,1.87907,1.23264,1.78203,1.39067,5.05372,1.21132,1.22675,1.26204,...,1.25542,1.26865,1.23117,1.22602,1.26204,1.20838,1.28923,1.23117,1.55678,3.31349
1,2.45351,0.29548,0.524904,0.76957,0.820438,0.501286,1.95144,0.643146,0.374127,0.332231,...,0.132304,0.245498,0.122014,0.123484,0.229327,0.406468,0.272694,0.15362,0.515252,18.5329
2,45.6619,8.77692,0.352876,9.52885,0.301048,0.182286,0.830023,8.87027,0.391033,0.261669,...,0.244763,9.29291,7.29511,432.474,17.8802,11.8184,8.39397,12.4829,7.67144,34.8975
3,29.8736,1.76994,2.91233,2.95259,2.88476,0.499081,3.66446,1.79787,0.905549,1.08122,...,2.40426,5.22528,1.69203,1.64645,1.95884,3.54355,2.11687,2.17273,3.97207,87.6148
4,2.94598,0.251378,0.299945,0.321205,0.416835,0.237413,0.794703,0.371187,0.265344,0.249173,...,0.281514,0.649761,0.343991,0.342521,0.331496,0.967291,0.736494,0.560823,1.51047,25.523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716,2.62845,2.23888,3.30711,2.32929,3.40636,2.24697,9.0655,2.23594,2.28886,2.28592,...,2.29107,2.35869,2.17714,2.29915,2.25799,2.29327,2.34987,2.33444,2.25946,4.75634
2717,72.0779,0.31165,0.420143,0.794561,0.362801,0.260198,1.08315,0.746784,0.271224,0.288129,...,2.65932,1.78464,2.45204,3.67144,3.90151,7.67732,3.58986,2.55494,2.55715,145.463
2718,3.83756,0.424844,0.561294,0.739434,0.565705,0.357957,1.57175,0.660786,0.410143,0.46674,...,0.441014,0.984932,0.538037,0.558618,0.804851,1.04226,0.61448,0.785741,1.4186,20.319
2719,41.7104,5.20838,0.320897,6.23447,0.223856,0.127159,0.688742,6.19552,0.249908,0.191841,...,0.229327,12.8975,6.77178,5.90886,7.99633,729.93,5.73319,6.20434,6.02132,31.3965


Making the dataframe accessible by the other files

In [ ]:
%store final_dataframe

Stored 'final_dataframe' (DataFrame)
